In [1]:
#| default_exp server_functions

# Server

In [6]:
#| export
from nbs.core import say_hello, test_model, relpath, imread, imshow, imwrite

In [24]:
#| export
import cv2
import io
import numpy as np

class FastApi:
    async def file2np_img(image, convert4to3channels=True):
        contents = await image.read()
        nparr = np.fromstring(contents, np.uint8)
        im = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)
        if im.shape[2] == 4 and convert4to3channels:
            img = im[:,:,:3]
            mask = im[:,:,3][:,:,None] / 255
            im = (img * mask) + (np.ones_like(img)*255 * (1-mask))
            im = np.array(im, np.uint8)
        im = im[:,:,::-1]
        return im
    
    def np_img2bytes(np_img):
        np_img = np_img[:,:,::-1]
        res, im_png = cv2.imencode(".png", np_img)
        return io.BytesIO(im_png.tobytes())

In [8]:
#| export
class Ping:
    def ping():
        return {"Hello": say_hello('world')}
    
    def ping_model():
        return {"Hello": test_model()}
    
    def ping_write_image():
        return cv2.imread(relpath('data/test.png'))
    
    def ping_read_image(np_img):
        return {"Hello": str(np_img.shape)}

In [13]:
#| export
from nbs.fastai_utils import ExamplePredictor
class PingFastai:
    def ping_fastai_pets(np_img):
        predictor = ExamplePredictor()
        predictor.load_learner('example_pets')
        return predictor.predict(np_img)

## Tests

In [14]:
from nbs.test_utils import test_eq, test_fail, test_close, Test, test_one_notebook, test_all_notebooks

In [15]:
with Test('Ping'):
    res = Ping.ping()
    test_eq(res['Hello'], 'Hi world!')

In [16]:
with Test('FastAI Ping'):
    with Test('pets'):
        random_image = imread('/app/data/example_pets/Egyptian_Mau_1.jpg')
        res = PingFastai.ping_fastai_pets(random_image)
        test_close(res[0]['score'], 1.0, eps=1.0)

In [18]:
#| notest
from nbs.test_utils import test_one_notebook
test_one_notebook('04_server_functions.ipynb')

Starting 04_server_functions.ipynb
- Completed 04_server_functions.ipynb
success: True duration: 0.9736499786376953


In [25]:
#| notest
from nbs.test_utils import test_all_notebooks
test_all_notebooks(do_print=True)

Starting /app/src/index.ipynb
- Completed /app/src/index.ipynb
Starting /app/src/03_test_utils.ipynb
- Completed /app/src/03_test_utils.ipynb
Starting /app/src/01_import_test.ipynb
- Completed /app/src/01_import_test.ipynb
Starting /app/src/02_gpu_tests.ipynb
- Completed /app/src/02_gpu_tests.ipynb
Starting /app/src/04_server_functions.ipynb
- Completed /app/src/04_server_functions.ipynb
Starting /app/src/05_fastai_utils.ipynb
- Completed /app/src/05_fastai_utils.ipynb
Starting /app/src/00_core.ipynb
- Completed /app/src/00_core.ipynb
Success.


In [26]:
!nbdev_export